In [1]:
%pylab qt
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Load the ggems files

In [2]:
# Loop through the ggems mhd files and save them to an array
import glob
import fastcat as fc
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
from mpl_interactions import ipyplot as iplt
# Zoom the ggems projections
from fastcat.patient_data import patient_phantom

base_directory = '/home/jericho/1-Workspace/3-jupyter_notebooks_patient/1-initial_patient_test/out_dir/'

ggems_scatter_files = glob.glob(base_directory + 'fastmc_*scatter.mhd')
ggems_primary_files = glob.glob(base_directory + 'fastmc_*[!scatter].mhd')
flood_file = base_directory + 'fastmc_00.0_flood.mhd'

ggems_scatter_files = [x for x in ggems_scatter_files if 'flood' not in x and '0_2' not in x]
ggems_primary_files = [x for x in ggems_primary_files if 'flood' not in x and '0_2' not in x]

ggems_scatter_files.sort()
ggems_primary_files.sort()

# Load the pickled phantom
# pkl_file = '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/test/fastmc_1e10_10angles.pkl'
# phantom = np.load(pkl_file, allow_pickle=True)

spectrum_file = '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/spectrum_120kVp_2.75mmAl_0.89mmTi.dat'

phantom = patient_phantom('brain', 1e10, is_fullfan=True)
phantom.spectrum_file = spectrum_file
angles = np.linspace(0, np.pi*2, 10, endpoint=False)
phantom.sim_angles = angles

# I flip the phantom because fastmc has a bit of a different coordinate system
phantom.phantom = np.flipud(phantom.phantom)
# flip the second axis left to right on the density phantom 
phantom.density = np.flip(phantom.density, axis=2)

# for ii, dens in enumerate(phantom.density):
#     dens = np.fliplr(dens)
# The density array is in fortran order, so I need to reorder the axes to c order 
# phantom.density = np.transpose(phantom.density, (2, 1, 0))

# phantom.simulation_parameters['angles'] = phantom.sim_angles

# Convert the phantom angles to degrees
phantom_angles = phantom.sim_angles * 180 / np.pi

# Get the simulation parameters
# simulation_parameters = phantom.simulation_parameters
detector_material = phantom.detector_material

# Load the spectrum as a fastcat spectrum object
# s = fc.Spectrum()
# s.load(spectrum_file=spectrum_file)
s = fc.Spekpy(100,14)
s.attenuate(0.089,fc.spectrum.get_mu('22'))
s.attenuate(0.4,fc.spectrum.get_mu('13'))

# s.x = np.array([59.99,60,60.01])
# s.y = np.array([0,1,0])
# plt.figure()
# plt.plot(s.x, s.y)
# phantom.geomet.nDetector = np.array([512,1024])
# phantom.geomet.dDetector = np.array([0.784, 0.392])
# phantom.geomet.sDetector = phantom.geomet.dDetector * phantom.geomet.nDetector

det = fc.Detector(s,'CsI-784-micrometer')
# angles = np.deg2rad([0,144,216,288,72])
angles = np.linspace(0, np.pi *2, 360, endpoint=False)
phantom.angles = angles
# phantom.bowtie_file = '/home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat'


[-229.5 -195.  -195. ]


Fastcat simulation

In [36]:
# return the projections
# phantom.phantom = np.flipud(phantom.phantom)
phantom.return_projs(det,s,angles,mgy=0,
                    ASG=False, scat_on=False,
                    det_on=False,
                    bowtie=True,
                    return_intensity=True)# Get primary fastcat projections

[2024-02-23 10:30:47,131] {ggems_simulate.py:228} INFO - Initializing filter /home/jericho/Software/fastcat/fastcat/data/ggems_files/full_fan_1_deg.dat


[2024-02-23 10:30:49,742] {ggems_simulate.py:591} INFO - Running Simulations
[2024-02-23 10:30:49,743] {ggems_simulate.py:625} INFO -     Skipping 10 keV as it has weight 2.2227754950378724e-13
[2024-02-23 10:30:49,744] {ggems_simulate.py:625} INFO -     Skipping 20 keV as it has weight 0.0006565983769606388
[2024-02-23 10:30:49,744] {ggems_simulate.py:629} INFO -     Simulating 30 keV
[2024-02-23 10:31:04,239] {ggems_simulate.py:629} INFO -     Simulating 40 keV
[2024-02-23 10:31:18,276] {ggems_simulate.py:629} INFO -     Simulating 50 keV
[2024-02-23 10:31:32,305] {ggems_simulate.py:629} INFO -     Simulating 60 keV
[2024-02-23 10:31:46,727] {ggems_simulate.py:629} INFO -     Simulating 70 keV
[2024-02-23 10:32:00,840] {ggems_simulate.py:629} INFO -     Simulating 80 keV
[2024-02-23 10:32:15,093] {ggems_simulate.py:629} INFO -     Simulating 90 keV
[2024-02-23 10:32:29,161] {ggems_simulate.py:625} INFO -     Skipping 100 keV as it has weight 0.006765526610445887
[2024-02-23 10:32:29,

In [3]:
# phantom.save_pickle('first_layer_915_brain_pre_recon.pkl')
# load the pickle file using the pickle module
import pickle

with open('first_layer_915_brain_pre_recon.pkl', 'rb') as f:
    phantom = pickle.load(f)

In [4]:
# phantom.zoom_factor = [1,1,2]

phantom.load_ggems_files(ggems_scatter_files, ggems_primary_files, flood_file,
                         average_scatter=False)

# phantom.detector_zoom = [1,2]
# phantom.zoom_ggems_files()

phantom.correct_intensity(ml=False)
# len(phantom.ggems_primary_projections)

In [5]:
# phantom.simulation_parameters['angles'] = np.linspace(0, np.pi *2, 2, endpoint=False)
phantom.interpolate_ggems_scatter()
phantom.add_poisson_noise(amplification=2)

In [6]:
det = fc.Detector(s,'CsI-392-micrometer')
# det.add_focal_spot(0.5)

# phantom.detector_mtf_convolution(det)

In [31]:
phantom.calc_ggems_projections(scat_on=True,scat_factor=0.5)

In [32]:
# Slice thickness = 0.995 mm

# Load the ct and cbct nrrd volumes using sitk 
import SimpleITK as sitk

ct_file = '/home/jericho/Documents/christian_patient_data/BRAIN/2_Stereo_CNS_Scan.nrrd'
cbct_file = '/home/jericho/Documents/christian_patient_data/BRAIN/2_Unnamed_Series.nrrd'

ct = sitk.ReadImage(ct_file)
cbct = sitk.ReadImage(cbct_file)

ct = sitk.GetArrayFromImage(ct)
cbct = sitk.GetArrayFromImage(cbct)

cbct = np.flip(cbct, axis=2)
# (0028, 0010) Rows                                US: 512
# (0028, 0011) Columns                             US: 512
# (0028, 0030) Pixel Spacing                       DS: [0.51119073851527, 0.51119073851527]
phantom.geomet.nVoxel = np.array(cbct.shape)
phantom.geomet.dVoxel = np.array([0.995,0.51119073851527,0.51119073851527])
phantom.geomet.sVoxel = phantom.geomet.nVoxel * phantom.geomet.dVoxel

phantom.reconstruct_ggems(algo='FDK',filt='ram_lak')

In [33]:
# fig = plt.figure(figsize=(10,5))
hi_res_bowtie_HU = phantom.img/np.mean(phantom.img[130:160,220:-220, 220:-220])
hi_res_bowtie_HU = (hi_res_bowtie_HU-(1) + np.mean(cbct[70:80,220:-220, 220:-220])/1000)*1000

# scale the values below -500 quadratically towards -1000
hi_res_bowtie_HU[hi_res_bowtie_HU < -500] = -1000 + (hi_res_bowtie_HU[hi_res_bowtie_HU < -500] + 1000)**(2)/500

# Make a circular mask about the center of the image and set all values outside a radius of 300 to -1000
x, y, z = np.ogrid[:hi_res_bowtie_HU.shape[0], :hi_res_bowtie_HU.shape[1], :hi_res_bowtie_HU.shape[2]]
mask = (x - hi_res_bowtie_HU.shape[0] / 2) ** 2 + (y - hi_res_bowtie_HU.shape[1] / 2) ** 2 + (z - hi_res_bowtie_HU.shape[2] / 2) ** 2 > 220 ** 2
hi_res_bowtie_HU[mask] = -1000

w = 355
l = 0

In [38]:
w, l = 1500, 200
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

ind_hline = 256
ind_vline = 256 - 70
ind_hline_rec = 256

ind_vline_rec = 256 - 70 + 6

crops = [40,40,40,40]

plt.figure(figsize=(6,6))
# plt.subplot(311)
# plt.imshow(np.fliplr(ct[139,125:-90,100:-100]), cmap='gray',vmin=l-w/2, vmax=l+w/2)
# plt.axis('off')
# plt.title('CT')
# Add a colorbar with a lable
# plt.colorbar(label='Hounsfield Units')
plt.subplot(223)
plt.imshow(cbct[79,crops[0]:-crops[1],crops[2]:-crops[3]], cmap='gray',vmin=l-w/2, vmax=l+w/2)
plt.axvline(ind_vline - crops[2], color='forestgreen', linestyle='--')
plt.axhline(ind_hline - crops[0], color='coral', linestyle='-.')

plt.axis('off')
plt.title('(c) OBI')

# Add an inset axis to the image of the top left corner 
# ax_inset = plt.axes([0.3, 0.6, 0.1, 0.1])
# ax_inset.imshow(cbct[79,70:150,220:-220], cmap='gray',vmin=l-w/2, vmax=l+w/2)
# ax_inset.axis('off')
ax = plt.gca()
# Do the same thing but using zoomed_inset_axes 
ax_inset = zoomed_inset_axes(ax, 2, loc='lower right')
ax_inset.imshow(cbct[79,crops[0]:-crops[1],crops[2]:-crops[3]], cmap='gray',vmin=l-w/2, vmax=l+w/2)
ax_inset.set_ylim(150-40,70-40)
ax_inset.set_xlim(220-40,512-220-40)
ax_inset.axis('off')
# Mark the region of the inset axis on the main image
mark_inset(ax, ax_inset, loc1=1, loc2=3, fc="none", ec="0.5")

# Indicate the region of the inset axis on the main image 
# plt.plot([70,150,150,70,70],[220,220,512-220,512-220,220], color='r')

plt.subplot(221)
plt.imshow(hi_res_bowtie_HU[82,crops[0]:-crops[1],crops[2]:-crops[3]], cmap='gray',vmin=l-w/2, vmax=l+w/2)
plt.axvline(ind_vline_rec - crops[2], color='cornflowerblue', linestyle='--')
plt.axhline(ind_hline_rec - crops[0], color='indianred', linestyle='-.')
plt.axis('off')
plt.title('(a) DFCIsim')
ax = plt.gca()
# Do the same thing but using zoomed_inset_axes 
ax_inset = zoomed_inset_axes(ax, 2, loc='lower right')
ax_inset.imshow(hi_res_bowtie_HU[82,crops[0]:-crops[1],crops[2]:-crops[3]], cmap='gray',vmin=l-w/2, vmax=l+w/2)
ax_inset.set_ylim(150-40,70-40)
ax_inset.set_xlim(220-40,512-220-40)
ax_inset.axis('off')
# Mark the region of the inset axis on the main image
mark_inset(ax, ax_inset, loc1=1, loc2=3, fc="none", ec="0.5")
plt.subplot(222)
plt.title('(b) Horizontal profile')
# plot a profile through the center of the images
# plt.plot(ct[139,ind_hline,100:-100], label='CT', color='black')
# xx = np.linspace()

nm=9
plt.plot(cbct[79,ind_hline-nm:ind_hline+nm,20:-6-20].mean(axis=0), label='CBCT', color='coral',lw=1)
plt.plot(hi_res_bowtie_HU[82,ind_hline_rec - nm:ind_hline_rec+nm,20+6:-20].mean(axis=0), label='DFCIsim', color='indianred',lw=1)
# Plot a red dotted line at the center of the image
plt.xlabel('Pixel number')
plt.ylabel('HU')
plt.grid('on')
plt.legend()

plt.subplot(224)
plt.title('(d) Vertical profile')
# plot a profile through the center of the images
# plt.plot(ct[139,125+30:-30-90,ind_vline], label='CT', color='black')
plt.plot(cbct[79,30:-30,ind_vline-nm:ind_vline+nm].mean(axis=1), label='CBCT', color='forestgreen',lw=1)
plt.plot(hi_res_bowtie_HU[82,30:-30,ind_vline_rec - nm:ind_vline_rec+nm].mean(axis=1), label='DFCIsim', color='cornflowerblue',lw=1)
# plt.axvline(256, color='r', linestyle='--')
plt.legend()
# plt.title('(d) Horizontal profile')
plt.xlabel('Pixel number')
plt.ylabel('HU')
plt.grid('on')
plt.tight_layout()

# print the difference between the cbct and the fastcat reconstructions along the horizontal and vertical profiles
print(100*np.mean(np.abs(cbct[79,30:-30,ind_vline-nm:ind_vline+nm].mean(axis=1) 
                     - hi_res_bowtie_HU[82,30:-30,ind_vline_rec - 
                                        nm:ind_vline_rec+nm].mean(axis=1))/
                                        np.max(cbct[79,30:-30,ind_vline-nm:ind_vline+nm].mean(axis=1))))
print(100*np.mean(np.abs(cbct[79,ind_hline-nm:ind_hline+nm,20:-6-20].mean(axis=0)
                        - hi_res_bowtie_HU[82,ind_hline_rec - nm:ind_hline_rec+nm,20+6:-20].mean(axis=0))/
                                            np.max(cbct[79,ind_hline-nm:ind_hline+nm,20:-6-20].mean(axis=0))))
# define these as variables
mean_diff_hor = 100*np.mean(np.abs(cbct[79,30:-30,ind_vline-nm:ind_vline+nm].mean(axis=1)
                        - hi_res_bowtie_HU[82,30:-30,ind_vline_rec - 
                                            nm:ind_vline_rec+nm].mean(axis=1))/
                                            np.max(cbct[79,30:-30,ind_vline-nm:ind_vline+nm].mean(axis=1)))
mean_diff_ver = 100*np.mean(np.abs(cbct[79,ind_hline-nm:ind_hline+nm,20:-6-20].mean(axis=0)
                        - hi_res_bowtie_HU[82,ind_hline_rec - nm:ind_hline_rec+nm,20+6:-20].mean(axis=0))/
                                            np.max(cbct[79,ind_hline-nm:ind_hline+nm,20:-6-20].mean(axis=0)))

print(np.mean([mean_diff_hor, mean_diff_ver]))
plt.savefig('cbct_fastmc_comparison.png', dpi=300)

/tmp/ipykernel_121233/4100745161.py:86: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


3.3011156831618553
4.313397493568148
3.807256588365002


In [33]:
print(np.mean([3.7789958727887303,
4.112039156004144]))

3.945517514396437


In [47]:
fig = plt.figure(dpi=300)
# plt.subplot(131)
# cc = pli.hyperslicer(phantom.img,vmin=0,vmax=0.059)
plt.subplot(121)
c2 = pli.hyperslicer(phantom.density[40:-40],autoscale_cmap=False,cmap='gray')
plt.colorbar(label='Density (g/cm$^3$)',orientation='horizontal')
plt.axis('off')
plt.subplot(122)
c3 = pli.hyperslicer(phantom.phantom[40:-40],autoscale_cmap=False,cmap='turbo',controls=c2)
plt.colorbar(label='Material Number',orientation='horizontal')
plt.axis('off')

(-0.5, 511.5, 466.5, -0.5)

In [50]:
c2.save_animation('brain_material_decomposition.gif',fig = fig,param='axis0',interval=50)

[2024-02-20 17:18:21,481] {animation.py:1060} INFO - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
[2024-02-20 17:18:21,482] {animation.py:338} INFO - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1920x998 -pix_fmt rgba -framerate 20.0 -loglevel error -i pipe: -filter_complex 'split [a][b];[a] palettegen [p];[b][p] paletteuse' -y brain_material_decomposition.gif


In [24]:
phantom.save_pickle('first_layer_360_brain.pkl')

## Now the second layer

In [146]:
spectrum_file = phantom.spectrum_file

s.load(spectrum_file=spectrum_file)
plt.figure()
plt.plot(s.x, s.y)
# s.attenuate(0.08,fc.spectrum.get_mu('CsI'))
s.attenuate(0.08,fc.spectrum.get_mu('CsI'))
s.attenuate(0.1,fc.spectrum.get_mu('14'))
plt.plot(s.x, s.y)

In [147]:
ggems_scatter_files = glob.glob(
    '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/out_dir/fastmc_*scatter.mhd'
)
ggems_primary_files = glob.glob(
    '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/out_dir/fastmc_*[!scatter].mhd'
)

ggems_scatter_files2 = [x for x in ggems_scatter_files if 'flood' not in x and '0_2' in x]
ggems_primary_files2 = [x for x in ggems_primary_files if 'flood' not in x and '0_2' in x]

ggems_scatter_files2.sort()
ggems_primary_files2.sort()

In [150]:
phantom2 = np.load(pkl_file, allow_pickle=True)
phantom2.DSD = 1515

# I flip the phantom because fastmc has a bit of a different coordinate system
phantom2.phantom = np.flipud(phantom2.phantom)

# phantom2.simulation_parameters['angles'] = phantom2.simangles

# Convert the phantom angles to degrees
phantom_angles = phantom2.sim_angles * 180 / np.pi

angles = np.linspace(0, np.pi *2, 360, endpoint=False)
phantom2.angles = angles
phantom2.bowtie_file = '/home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat'
phantom2.bowtie_interp = 'cubic'

In [ ]:
# plt.figure()
# cc =pli.hyperslicer(phantom.phantom)

# crop_min = [0,45,0]
# crop_max = [None, None, None]
# phantom2 = phantom.phantom[crop_min[0]:crop_max[0],crop_min[1]:crop_max[1],crop_min[2]:crop_max[2]]

# plt.figure()
# # Plot the three views of the phantom
# plt.subplot(131)
# plt.imshow(phantom2[phantom2.shape[0]//2], cmap='gray')
# # plot a star at the center of the slice
# plt.axis('off')
# plt.plot(phantom2.shape[2]//2, phantom2.shape[1]//2, 'r*')
# plt.subplot(132)
# plt.imshow(phantom2[:,phantom2.shape[1]//2], cmap='gray')
# # plot a star at the center of the slice
# plt.plot(phantom2.shape[2]//2, phantom2.shape[0]//2, 'r*')
# plt.axis('off')

# plt.subplot(133)
# plt.imshow(phantom2[:,:,phantom2.shape[2]//2], cmap='gray')
# # plot a star at the center of the slice
# plt.plot(phantom2.shape[1]//2, phantom2.shape[0]//2, 'r*')
# plt.axis('off')
# plt.tight_layout()

In [151]:
# return the projections
# phantom.phantom = np.flipud(phantom.phantom)
phantom2.return_projs(det,s,angles,mgy=0,
                    ASG=False, scat_on=False,
                    det_on=False,
                    bowtie=True,
                    return_intensity=True)# Get primary fastcat projections

[2024-02-08 09:52:39,228] {ggems_simulate.py:227} INFO - Initializing filter /home/jericho/Software/Catphan604_HeadSized/full_fan_1_deg.dat


[2024-02-08 09:52:40,122] {ggems_simulate.py:618} INFO - Running Simulations
[2024-02-08 09:52:40,122] {ggems_simulate.py:621} INFO -     1.0 mm focal spot added
[2024-02-08 09:52:40,123] {ggems_simulate.py:646} INFO -     Simulating 20 keV
[2024-02-08 09:52:52,132] {ggems_simulate.py:646} INFO -     Simulating 30 keV
[2024-02-08 09:53:03,800] {ggems_simulate.py:646} INFO -     Simulating 40 keV
[2024-02-08 09:53:15,325] {ggems_simulate.py:646} INFO -     Simulating 50 keV
[2024-02-08 09:53:26,728] {ggems_simulate.py:646} INFO -     Simulating 60 keV
[2024-02-08 09:53:38,153] {ggems_simulate.py:646} INFO -     Simulating 70 keV
[2024-02-08 09:53:49,633] {ggems_simulate.py:646} INFO -     Simulating 80 keV
[2024-02-08 09:54:01,264] {ggems_simulate.py:646} INFO -     Simulating 90 keV
[2024-02-08 09:54:12,958] {ggems_simulate.py:646} INFO -     Simulating 100 keV
[2024-02-08 09:54:24,454] {ggems_simulate.py:876} INFO - Weighting simulations
[2024-02-08 09:54:24,457] {ggems_simulate.py:96

In [152]:
flood_file2 = '/home/jericho/1-Workspace/1-jupyter_notebooks/4-preliminary_fastmc_projections/out_dir/fastmc_00.0_flood_2.mhd'

phantom2.load_ggems_files(ggems_scatter_files2, ggems_primary_files2, flood_file2,
                         average_scatter=True)
phantom2.correct_intensity(ml=False)

In [155]:
plt.figure()
plt.subplot(131)

from scipy.ndimage import gaussian_filter
filt_sigma = 1
int1 = gaussian_filter(phantom.ggems_primary_projections[0],filt_sigma)
int2 = gaussian_filter(phantom2.ggems_primary_projections[0],filt_sigma)

proj1 = -np.log(int1/np.rot90(phantom.ggems_flood))
proj2 = -np.log(int2/np.rot90(phantom2.ggems_flood))
# gaussian blur the projections using scipy ndimage 

crop = 20

plt.imshow(proj1[crop:-crop,crop:-crop],cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.title(f'GGEMS $\sigma$={filt_sigma}')
plt.subplot(132)
plt.imshow(proj2[crop:-crop,crop:-crop],cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.title(f'Fastcat $\sigma$={filt_sigma}')
# Plot the difference 
plt.subplot(133)
diff_1 = 100*(proj1[crop:-crop,crop:-crop]-proj2[crop:-crop,crop:-crop])/4
plt.axis('off')
plt.title('Difference')
plt.imshow(diff_1,cmap='bwr',vmin=-5,vmax=5)
plt.colorbar(orientation='horizontal')

# plot the mean of the difference along the x and y axes
plt.figure()
plt.plot(np.mean(diff_1,axis=0))
plt.plot(np.mean(diff_1,axis=1))

/tmp/ipykernel_6155/2390269776.py:9: RuntimeWarning: divide by zero encountered in divide
  proj1 = -np.log(int1/np.rot90(phantom.ggems_flood))
/tmp/ipykernel_6155/2390269776.py:10: RuntimeWarning: divide by zero encountered in divide
  proj2 = -np.log(int2/np.rot90(phantom2.ggems_flood))


In [171]:
plt.figure()
plt.subplot(131)

from scipy.ndimage import gaussian_filter
filt_sigma = 1
int1 = gaussian_filter(phantom.intensity[0],filt_sigma)
int2 = gaussian_filter(phantom2.intensity[0],filt_sigma)

proj1 = -np.log(int1/phantom.flood_field)
proj2 = -np.log(int2/phantom2.flood_field)
# gaussian blur the projections using scipy ndimage 

crop = 20

plt.imshow(proj1[crop:-crop,crop:-crop],cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.title(f'Layer 1 norm. filt. $\sigma$={filt_sigma}')
plt.subplot(132)
plt.imshow(proj2[crop:-crop,crop:-crop],cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.title(f'Layer 2 norm. filt. $\sigma$={filt_sigma}')
# Plot the difference 
plt.subplot(133)
diff_1 = 100*(proj1[crop:-crop,crop:-crop]-proj2[crop:-crop,crop:-crop])/4
plt.axis('off')
plt.title('Difference')
plt.imshow(diff_1,cmap='bwr',vmin=-10,vmax=10)
plt.colorbar(orientation='horizontal')

# plot the mean of the difference along the x and y axes
plt.figure()
plt.plot(np.mean(diff_1,axis=0))
plt.plot(np.mean(diff_1,axis=1))

In [156]:
plt.figure()
plt.subplot(131)

from scipy.ndimage import gaussian_filter
filt_sigma = 1
int1 = gaussian_filter(phantom2.ggems_primary_projections[0],filt_sigma)
int2 = gaussian_filter(phantom2.intensity[0],filt_sigma)

proj1 = -np.log(int1/np.rot90(phantom2.ggems_flood))
proj2 = -np.log(int2/phantom2.flood_field)
# gaussian blur the projections using scipy ndimage 

crop = 20

plt.imshow(proj1[crop:-crop,crop:-crop],cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.title(f'GGEMS $\sigma$={filt_sigma}')
plt.subplot(132)
plt.imshow(proj2[crop:-crop,crop:-crop],cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.title(f'Fastcat $\sigma$={filt_sigma}')
# Plot the difference 
plt.subplot(133)
diff_1 = 100*(proj1[crop:-crop,crop:-crop]-proj2[crop:-crop,crop:-crop])/4
plt.axis('off')
plt.title('Difference')
plt.imshow(diff_1,cmap='bwr',vmin=-5,vmax=5)
plt.colorbar(orientation='horizontal')

# plot the mean of the difference along the x and y axes
plt.figure()
plt.plot(np.mean(diff_1,axis=0))
plt.plot(np.mean(diff_1,axis=1))

/tmp/ipykernel_6155/1860078263.py:9: RuntimeWarning: divide by zero encountered in divide
  proj1 = -np.log(int1/np.rot90(phantom2.ggems_flood))


In [157]:
plt.figure(figsize=(10,5))
# plt.subplot(121)
plt.plot(np.mean(phantom2.ggems_primary_projections[0],axis=0))
plt.plot(np.mean(phantom2.ggems_flood,axis=1))
# plt.subplot(122)
plt.plot(np.mean(phantom2.intensity[0],axis=0))
plt.plot(np.mean(phantom2.flood_field,axis=0))

In [158]:
# phantom2.simulation_parameters['angles'] = np.linspace(0, np.pi *2, 2, endpoint=False)
phantom2.interpolate_ggems_scatter()
phantom2.add_poisson_noise()

In [159]:
plt.figure()
plt.hist(phantom2.ggems_primary_projections[1].flatten(), bins=100,histtype='step',label='ggems')
plt.hist(phantom2.intensity[0].flatten(), bins=100,histtype='step',label='fastcat')
plt.hist(phantom.ggems_primary_projections[1].flatten(), bins=100,histtype='step',label='ggems_1')
plt.hist(phantom.intensity[0].flatten(), bins=100,histtype='step',label='fastcat_1')
plt.legend()

# Plot the same thing but normalized
plt.figure()
plt.hist(phantom2.ggems_primary_projections[1].flatten()/phantom2.ggems_primary_projections[1].max(), bins=100,histtype='step',label='ggems')
plt.hist(phantom2.intensity[0].flatten()/phantom2.intensity[0].max(), bins=100,histtype='step',label='fastcat')
plt.hist(phantom.ggems_primary_projections[1].flatten()/phantom.ggems_primary_projections[1].max(), bins=100,histtype='step',label='ggems_1')
plt.hist(phantom.intensity[0].flatten()/phantom.intensity[0].max(), bins=100,histtype='step',label='fastcat_1')
plt.legend()


In [365]:
plt.figure()
plt.hist(phantom.proj[0].flatten(), bins=100,histtype='step',label='fastcat')
plt.hist(phantom2.proj[0].flatten(), bins=100,histtype='step',label='fastcat_2')

(array([93135.,  2662.,  2453.,   407.,  1183.,   336.,   343.,   373.,
          823.,   673.,  2308.,   407.,   325.,  1160.,   282.,   403.,
         1047.,   370.,   308.,  1115.,   343.,  1076.,   349.,   921.,
          489.,   999.,   389.,  1029.,   618.,   769.,   933.,   794.,
          653.,  1105.,   990.,   463.,   904.,   993.,  1103.,   962.,
         1054.,   959.,   862.,  1070.,  1034.,  1159.,  1000.,  1067.,
         1204.,  1147.,  1197.,  1097.,  1131.,  1361.,  1161.,  1315.,
         1190.,  1362.,  1461.,  1532.,  1470.,  1347.,  1510.,  1586.,
         1570.,  1452.,  1680.,  1357.,  1500.,  1680.,  1551.,  1842.,
         1980.,  1804.,  1941.,  1965.,  1869.,  1868.,  1811.,  2157.,
         2258.,  2070.,  1750.,  1851.,  3150.,  4800.,  3324.,  3911.,
         4416.,  5734.,  4236.,  5188.,  3527.,  3567.,  4934., 11401.,
         8570.,  1264.,  1195.,  1700.]),
 array([ 0.16441574,  0.51807065,  0.87172556,  1.22538047,  1.57903538,
         1.93269029, 

In [160]:
det = fc.Detector(s,'CsI-784-micrometer')
det.add_focal_spot(1.0)

phantom2.detector_mtf_convolution(det)

In [161]:
phantom2.calc_ggems_projections(scat_on=True)
phantom2.reconstruct_ggems(algo='FDK')

In [162]:
fig = plt.figure(figsize=(10,5))
hi_res_bowtie_HU = phantom2.img/np.mean(phantom2.img[20:-20,200:-200, 200:-200])
hi_res_bowtie_HU = (hi_res_bowtie_HU-1)*1000
cont = pli.hyperslicer(hi_res_bowtie_HU, autoscale_cmap=False,cmap='bone', vmin=-400, vmax=400)#,controls=cc)
plt.colorbar()
plt.title('High res bowtie [HU]')
plt.axis('off')

(-0.5, 525.5, 525.5, -0.5)

In [166]:
fig = plt.figure(figsize=(10,5))
plt.subplot(132)

hi_res_bowtie_HU = phantom2.img/np.mean(phantom.img[20:-20,200:-200, 200:-200])
hi_res_bowtie_HU = (hi_res_bowtie_HU-1)*1000
cont = pli.hyperslicer(hi_res_bowtie_HU, autoscale_cmap=False,cmap='bone', vmin=-400, vmax=400,play_buttons=True)#,controls=cc)
# plt.colorbar()
plt.title('Second Layer')
plt.axis('off')
plt.subplot(131)
first_layer_HU = phantom.img/np.mean(phantom.img[20:-20,200:-200, 200:-200])
first_layer_HU = (first_layer_HU-1)*1000
cc = pli.hyperslicer(first_layer_HU, autoscale_cmap=False,cmap='bone', vmin=-400, vmax=400,controls=cont)
# plt.colorbar()
plt.title('First Layer')
plt.axis('off')
plt.subplot(133)
# Plot the difference
diff = hi_res_bowtie_HU - first_layer_HU
cc = pli.hyperslicer(diff, autoscale_cmap=False,cmap='bwr', vmin=-400, vmax=400,controls=cont)
plt.colorbar()
plt.title('Difference')
plt.tight_layout()
plt.axis('off')


(-0.5, 525.5, 525.5, -0.5)

In [43]:
cont.save_animation('catphan_correct_spectrum.gif',fig = fig,param='axis0')

NameError: name 'cont' is not defined

In [163]:
plt.figure()
plt.plot(phantom.weights_fluence)
plt.plot(phantom2.weights_fluence)

In [164]:
# Save the flood fields, the raw projections, and the scatter all to matlab files 

# Save the flood fields
import scipy.io as sio

sio.savemat('flood_field_first_layer.mat', {'flood_fields': phantom.flood_field})
sio.savemat('flood_field_second_layer.mat', {'flood_fields': phantom2.flood_field})

# Save the intensity
sio.savemat('detector_intensity_first_layer.mat', {'intensity': phantom.intensity})
sio.savemat('detector_intensity_second_layer.mat', {'intensity': phantom2.intensity})

# Save the scatter
sio.savemat('detector_scatter_intensity_first_layer.mat', {'scatter': phantom.interpolated_ggems_scatter})
sio.savemat('detector_scatter_intensity_second_layer.mat', {'scatter': phantom2.interpolated_ggems_scatter})


In [165]:
phantom2.save_pickle('second_layer_360.pkl')
phantom.save_pickle('first_layer_360.pkl')

In [ ]:
plt.figure()
plt.subplot(131)

from scipy.ndimage import gaussian_filter
filt_sigma = 0.1
int1 = gaussian_filter(phantom.ggems_primary_projections[0],filt_sigma)
int2 = gaussian_filter(phantom.intensity[0],filt_sigma)

proj1 = -np.log(int1/np.rot90(phantom.ggems_flood))
proj2 = -np.log(int2/np.rot90(phantom.flood_field))
# gaussian blur the projections using scipy ndimage 

crop = 20

plt.imshow(proj1[crop:-crop,crop:-crop],cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.title(f'GGEMS $\sigma$={filt_sigma}')
plt.subplot(132)
plt.imshow(proj2[crop:-crop,crop:-crop],cmap='turbo',vmin=0,vmax=4)
plt.axis('off')
plt.title(f'Fastcat $\sigma$={filt_sigma}')
# Plot the difference 
plt.subplot(133)
diff_1 = 100*(proj1[crop:-crop,crop:-crop]-proj2[crop:-crop,crop:-crop])/4
plt.axis('off')
plt.title('Difference')
plt.imshow(diff_1,cmap='bwr',vmin=-5,vmax=5)
plt.colorbar(orientation='horizontal')

# plot the mean of the difference image along both axes
plt.figure()
plt.plot(np.mean(diff_1,axis=0))
plt.plot(np.mean(diff_1,axis=1))

/tmp/ipykernel_373177/4284414012.py:9: RuntimeWarning: divide by zero encountered in divide
  proj1 = -np.log(int1/np.rot90(phantom.ggems_flood))
/tmp/ipykernel_373177/4284414012.py:9: RuntimeWarning: invalid value encountered in divide
  proj1 = -np.log(int1/np.rot90(phantom.ggems_flood))
/tmp/ipykernel_373177/4284414012.py:9: RuntimeWarning: divide by zero encountered in log
  proj1 = -np.log(int1/np.rot90(phantom.ggems_flood))
